# NC 다이노스 선수 데이터 분석
## 1. 데이터 적재 및 통합

In [1]:
# 라이브러리 가져오기
import pandas as pd

In [2]:
# 데이터 가져오기
# 파일명 : NC Dinos.xlsx
# 세 개의 시트에 데이터가 저장되어 있음
data = pd.read_excel('data/NC Dinos.xlsx', sheet_name=None)
nc13, nc14, nc15 = data.values()

In [6]:
nc13.drop('Unnamed: 0', axis=1, inplace=True)
nc14.drop('Unnamed: 0', axis=1, inplace=True)
nc14.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
nc13.shape

(20, 19)

In [8]:
nc14.shape

(20, 19)

In [10]:
nc15.shape

(20, 19)

In [11]:
nc15.head(2)

,선수명,팀명,경기,타석,타수,안타,홈런,득점,타점,볼넷,삼진,도루,BABIP,타율,출루율,장타율,OPS,wOBA,WAR
0,테임즈,NC,142,595,472,180,47,130,140,103,91,40,0.390,0.381,0.497,0.790,1.287,0.52,11.73
1,나성범,NC,144,622,564,184,28,112,135,33,127,23,0.374,0.326,0.373,0.553,0.926,0.39,4.46


In [13]:
# 단, 각 시트의 원본 데이터가 몇 년도 데이터인지 알 수 있도록 '연도' 값을 추가
# 컬럼 추가 : 연도
# 값 : 2013, 2014, 2015 
# 스칼라값으로 컬럼 추가
nc13['연도'] = 2013
nc13.head(2)

,선수명,팀명,경기,타석,타수,안타,홈런,득점,타점,볼넷,삼진,도루,BABIP,타율,출루율,장타율,OPS,wOBA,WAR,연도
0,모창민,NC,108,436,395,109,12,57,51,37,68,16,0.307,0.276,0.339,0.443,0.782,0.353,2.31,2013
1,이호준,NC,126,508,442,123,20,46,87,60,109,2,0.324,0.278,0.362,0.475,0.837,0.373,1.85,2013


In [14]:
nc14['연도'] = 2014
nc15['연도'] = 2015

In [ ]:
# 전체 데이터를 하나의 데이터프레임으로 통합
# 최종 데이터프레임 구조 : (60,19)

In [18]:
nc14.columns.sort_values() == nc13.columns.sort_values()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [22]:
set(nc14['선수명'].values) - set(nc13['선수명'].values)

{'강민국', '권희동', '김준완', '김태우', '노진혁', '박명환*', '손시헌', '이종욱', '최재원', '테임즈', '허준'}

In [25]:
# append
df1 = nc13.append([nc14, nc15], ignore_index=True)

In [26]:
# concat
df2 = pd.concat([nc13, nc14, nc15], ignore_index=True)

In [27]:
df1.shape

(60, 20)

In [28]:
df2.shape

(60, 20)

In [31]:
df1[df1['연도'] == 2013].shape

(20, 20)

## 2. 2015년 타율이 높은 상위 10명의 이름
#### 참고) 동일 타율을 기록한 선수에 대해서는 안타 개수가 많은 순서로 정렬


In [36]:
t1 = df1[df1['연도']==2015].sort_values(['타율','안타'], ascending=False).head(10)

In [42]:
# list(t1['선수명'].values)
t1['선수명'].tolist()

['테임즈', '박정준', '조평호', '나성범', '박민우', '김종호', '이호준', '모창민', '용덕한', '조영훈']

## 3.  2013~2015년  선수별 3년 평균 안타 수

- 평균 안타수가 높은 순서로 정렬해서 보이기

In [68]:
# pivot_table 사용할 경우
# 로우 : 선수명
# 값 : 안타
# 집계함수 : 평균
df1.pivot_table(index='선수명', values='안타', aggfunc='mean').sort_values('안타', ascending=False).head(10)

,안타
선수명,
테임즈,166.000000
나성범,146.333333
김종호,127.000000
이호준,123.333333
이종욱,122.000000
김태군,107.000000
손시헌,99.000000
박민우,97.666667
모창민,93.666667


In [71]:
# 평균 안타 상위 10명에 대한 선수이름
names = df1.pivot_table(index='선수명',
                        values='안타', 
                        aggfunc='mean')
.sort_values('안타',ascending=False).head(10).index.tolist()

In [48]:
# pivot_table 사용하지 않는 경우
# 인덱스 : 연도, 선수명 컬럼을 계층 인덱스로 적용
tmp1 = df1.set_index(['연도','선수명']).copy()

In [58]:
# 3개년 평균 안타
tmp1.head(3)
# 연도별(컬럼), 선수별(로우) 안타수를 보기 위한 구조 변경
tmp1['안타'].unstack(0)
# 각 선수의 3개년 안타 평균
tmp1['안타'].unstack(0).mean(axis=1)
# 내림차순 정렬 & 상위 10
tmp1['안타'].unstack(0).mean(axis=1).sort_values(ascending=False).head(10)

선수명
테임즈    166.000000
나성범    146.333333
김종호    127.000000
이호준    123.333333
이종욱    122.000000
김태군    107.000000
손시헌     99.000000
박민우     97.666667
모창민     93.666667
지석훈     79.000000
dtype: float64

## 5. 평균 안타수 기준 상위 10위 타자들의 3년간 안타, 홈런, 타율 조회

#### 참고) NaN 값이 있을 경우 '-'로 대체해서 표현 => df.fillna('-')

In [63]:
# df1.pivot_table(index='선수명', values='타율')
# df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 20 columns):
선수명      60 non-null object
팀명       60 non-null object
경기       60 non-null int64
타석       60 non-null int64
타수       60 non-null int64
안타       60 non-null int64
홈런       60 non-null int64
득점       60 non-null int64
타점       60 non-null int64
볼넷       60 non-null int64
삼진       60 non-null int64
도루       60 non-null int64
BABIP    60 non-null object
타율       60 non-null object
출루율      60 non-null float64
장타율      60 non-null object
OPS      60 non-null object
wOBA     60 non-null float64
WAR      60 non-null float64
연도       60 non-null int64
dtypes: float64(3), int64(11), object(6)
memory usage: 9.5+ KB


In [64]:
test = pd.DataFrame({'a':[1,2,3],'b':[4,5,6]}, index=['A','B','C'])
test

,a,b
A,1,4
B,2,5
C,3,6


In [66]:
# 라벨 인덱스에 대한 멀티 인덱싱
test.loc[['A','C']]

,a,b
A,1,4
C,3,6


In [73]:
# 1. 전체 df에서 필요한 값만 추출
# 선수명, 연도, 타율, 홈런, 안타
df3 = df1[['선수명', '연도', '타율', '홈런', '안타']].copy()

In [78]:
# 선수명 컬럼을 인덱스로 변경 => 라벨 인덱스에 대한 멀티 인덱싱
tmp = df3.set_index('선수명').loc[names].copy()

In [86]:
# 구조 재배열
tmp.reset_index().set_index(['연도','선수명']).unstack(0)

타율                  홈런                 안타              
연도    2013   2014   2015  2013  2014  2015   2013   2014   2015
선수명                                                            
김종호  0.277    NaN  0.295   0.0   NaN   4.0  129.0    NaN  125.0
김태군    NaN    NaN  0.254   NaN   NaN   6.0    NaN    NaN  107.0
나성범  0.243  0.329  0.326  14.0  30.0  28.0   98.0  157.0  184.0
모창민  0.276  0.263   0.29  12.0  16.0   6.0  109.0  110.0   62.0
박민우  0.262  0.298  0.304   0.0   1.0   3.0   11.0  124.0  158.0
손시헌    NaN  0.293  0.245   NaN   5.0  13.0    NaN   90.0  108.0
이종욱    NaN  0.288  0.268   NaN   6.0   5.0    NaN  126.0  118.0
이호준  0.278  0.271  0.294  20.0  23.0  24.0  123.0  115.0  132.0
지석훈  0.220  0.274  0.267   3.0   6.0  11.0   68.0   58.0  111.0
테임즈    NaN  0.343  0.381   NaN  37.0  47.0    NaN  152.0  180.0

In [100]:
#### 구조 먼저 잡고 선수명 추출
df1.set_index(['연도','선수명'])[['타율', '홈런', '안타']].unstack(0).loc[names]

타율                  홈런                 안타              
연도    2013   2014   2015  2013  2014  2015   2013   2014   2015
선수명                                                            
테임즈    NaN  0.343  0.381   NaN  37.0  47.0    NaN  152.0  180.0
나성범  0.243  0.329  0.326  14.0  30.0  28.0   98.0  157.0  184.0
김종호  0.277    NaN  0.295   0.0   NaN   4.0  129.0    NaN  125.0
이호준  0.278  0.271  0.294  20.0  23.0  24.0  123.0  115.0  132.0
이종욱    NaN  0.288  0.268   NaN   6.0   5.0    NaN  126.0  118.0
김태군    NaN    NaN  0.254   NaN   NaN   6.0    NaN    NaN  107.0
손시헌    NaN  0.293  0.245   NaN   5.0  13.0    NaN   90.0  108.0
박민우  0.262  0.298  0.304   0.0   1.0   3.0   11.0  124.0  158.0
모창민  0.276  0.263   0.29  12.0  16.0   6.0  109.0  110.0   62.0
지석훈  0.220  0.274  0.267   3.0   6.0  11.0   68.0   58.0  111.0

In [90]:
# 2. 구조 변경하지 않고 선수명 컬럼 값에 대해서 특정 선수명만 추출
tmp2 = df3[df3['선수명'].isin(names)].copy()

In [95]:
tmp2.set_index(['연도','선수명']).unstack(0).fillna('-')

타율                 홈런              안타            
연도    2013   2014   2015 2013 2014  2015 2013 2014   2015
선수명                                                      
김종호  0.277      -  0.295    0    -   4.0  129    -  125.0
김태군      -      -  0.254    -    -   6.0    -    -  107.0
나성범  0.243  0.329  0.326   14   30  28.0   98  157  184.0
모창민  0.276  0.263  0.290   12   16   6.0  109  110   62.0
박민우  0.262  0.298  0.304    0    1   3.0   11  124  158.0
손시헌      -  0.293  0.245    -    5  13.0    -   90  108.0
이종욱      -  0.288  0.268    -    6   5.0    -  126  118.0
이호준  0.278  0.271  0.294   20   23  24.0  123  115  132.0
지석훈  0.220  0.274  0.267    3    6  11.0   68   58  111.0
테임즈      -  0.343  0.381    -   37  47.0    -  152  180.0

## 6. 연도별 팀의 총 안타수, 총 홈런수, 총 도루수

In [115]:
# 로우 : 연도
# 값 : 안타, 홈런, 도루
# 집계함수 : 총합
t = df1.pivot_table(index='연도', values=['안타', '홈런', '도루'], aggfunc='sum').copy()

In [117]:
t['t'] = [1,2,3]

In [122]:
t.reset_index().set_index(['연도', 't']).unstack(0).reset_index(drop=True)

도루                   안타                    홈런              
연도   2013   2014   2015   2013    2014    2015  2013   2014   2015
0   133.0    NaN    NaN  814.0     NaN     NaN  63.0    NaN    NaN
1     NaN  118.0    NaN    NaN  1039.0     NaN   NaN  139.0    NaN
2     NaN    NaN  201.0    NaN     NaN  1380.0   NaN    NaN  158.0